<table class="ee-notebook-buttons" align="left">
    <td><a target="_parent"  href="https://github.com/gee-community/geemap/tree/master/tutorials/Image/04_math_operations.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_parent"  href="https://nbviewer.jupyter.org/github/gee-community/geemap/blob/master/tutorials/Image/04_math_operations.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_parent"  href="https://colab.research.google.com/github/gee-community/geemap/blob/master/tutorials/Image/04_math_operations.ipynb"><img width=26px src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mathematical operations
Earth Engine supports many basic mathematical operators. They share some common features. Earth Engine performs math operations per pixel. When an operator is applied to an image, it's applied to each unmasked pixel of each band. In the case of operations on two images, the operation is only applied at the locations where pixels in both images are unmasked. Earth Engine automatically matches bands between images. When an operator is applied to two images, the images are expected to have the same number of bands so they can be matched pairwise. However, if one of the images has only a single band, it is matched with all of the bands in the other image, essentially replicating that band enough times to match the other image.

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/gee-community/geemap). The **geemap** Python package is built upon the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages and implements several methods for interacting with Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its [dependencies](https://github.com/gee-community/geemap#dependencies), including earthengine-api, folium, and ipyleaflet.

**Important note**: A key difference between folium and ipyleaflet is that ipyleaflet is built upon ipywidgets and allows bidirectional communication between the front-end and the backend enabling the use of the map to capture user input, while folium is meant for displaying static data only ([source](https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a)). Note that [Google Colab](https://colab.research.google.com/) currently does not support ipyleaflet ([source](https://github.com/googlecolab/colabtools/issues/60#issuecomment-596225619)). Therefore, if you are using geemap with Google Colab, you should use [`import geemap.foliumap`](https://github.com/gee-community/geemap/blob/master/geemap/foliumap.py). If you are using geemap with [binder](https://mybinder.org/) or a local Jupyter notebook server, you can use [`import geemap`](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py), which provides more functionalities for capturing user input (e.g., mouse-clicking and moving).

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print("geemap package not installed. Installing ...")
    subprocess.check_call(["python", "-m", "pip", "install", "geemap"])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.foliumap as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
The default basemap is `Google Satellite`. [Additional basemaps](https://github.com/gee-community/geemap/blob/master/geemap/geemap.py#L13) can be added using the `Map.add_basemap()` function. 

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map.add_basemap("ROADMAP")  # Add Google Map
Map

## Add Earth Engine Python script 

For a simple example, consider the task of creating the Normalized Difference Vegetation Index (NDVI) using Landsat imagery:

In [ ]:
# Load two 5-year Landsat 7 composites.
landsat1999 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003")
landsat2008 = ee.Image("LANDSAT/LE7_TOA_5YEAR/2008_2012")

# Compute NDVI the hard way.
ndvi1999 = (
    landsat1999.select("B4")
    .subtract(landsat1999.select("B3"))
    .divide(landsat1999.select("B4").add(landsat1999.select("B3")))
)

# Compute NDVI the easy way.
ndvi2008 = landsat2008.normalizedDifference(["B4", "B3"])

# A nice NDVI palette.
palette = [
    "#d73027",
    "#f46d43",
    "#fdae61",
    "#fee08b",
    "#d9ef8b",
    "#a6d96a",
    "#66bd63",
    "#1a9850",
]

Map.addLayer(landsat1999, {"bands": ["B4", "B3", "B2"]}, "Landsat 1999")
Map.addLayer(landsat2008, {"bands": ["B4", "B3", "B2"]}, "Landsat 2008")
Map.addLayer(ndvi1999, {"palette": palette}, "NDVI 1999")
Map.addLayer(ndvi2008, {"palette": palette}, "NDVI 2008")

As shown in the previous example, math operators perform basic arithmetic operations on image bands. The normalized difference operation is so common in remote sensing, Earth Engine provides a shortcut method, as shown in the second part of the example. Subtracting the images in this example results in a “change vector” for each pixel. Bands are matched automatically to perform the difference:

In [ ]:
# Compute the multi-band difference image.
diff = landsat2008.subtract(landsat1999)
Map.addLayer(diff, {"bands": ["B4", "B3", "B2"], "min": -32, "max": 32}, "difference")

# Compute the squared difference in each band.
squaredDifference = diff.pow(2)
Map.addLayer(
    squaredDifference, {"bands": ["B4", "B3", "B2"], "max": 1000}, "squared diff."
)

In the second part of this example, the squared difference is computed using `image.pow(2)`. For the complete list of mathematical operators handling basic arithmetic, trigonometry, exponentiation, rounding, casting, bitwise operations and more, see the API documentation (in the **Docs** tab of the [Earth Engine Code Editor](https://code.earthengine.google.com/)).

In [ ]:
Map.addLayerControl()
Map

### Expressions
To implement more complex mathematical expressions, it may be more convenient to use `image.expression()`, which parses a text representation of a math operation. The following example uses `expression()` to compute the Enhanced Vegetation Index (EVI):

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)

In [ ]:
# Load a Landsat 8 image.
image = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318")

# Compute the EVI using an expression.
evi = image.expression(
    "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
    {"NIR": image.select("B5"), "RED": image.select("B4"), "BLUE": image.select("B2")},
)

Map.centerObject(image, 9)
Map.addLayer(image, {"bands": ["B5", "B4", "B3"]}, "Landsat 8")
Map.addLayer(evi, {"min": -1, "max": 1, "palette": ["FF0000", "00FF00"]}, "EVI")

## Display Earth Engine data layers 

In [ ]:
Map.addLayerControl()
Map

Observe that the first argument to expression is the textual representation of the math operation, the second argument is a dictionary where the keys are variable names used in the expression and the values are the image bands to which the variables should be mapped. Bands in the image may be referred to as `b("band name")` or `b(index)`, for example `b(0)`, instead of providing the dictionary. Note that division functions as it does in Python: dividing two integers results in an integer. For example `10 / 20 = 0`. To change this behavior, multiply one of the operands by `1.0: 10 * 1.0 / 20 = 0.5`. Supported expression operators are listed in the following table.

![](https://i.imgur.com/TyXM4s9.png)